In [1]:
import pandas as pd
from Constraint import Problem, Constraint
from Nature import attribute_nature

In [2]:
experiment_constraint_data = pd.read_csv('./experiment_constraint_data.csv')

In [3]:
experiment_constraint_data.head(10)

,name,value,priority,constraint_no
0,os,10,5,0
1,ram,16,2,0
2,screen_size,14.4,2,0
3,camera,True,5,0
4,cpu,ryzen 5|ryzen 5,3,0
5,weight,1.99,5,0
6,weight,1.76,3,1
7,price,1744.15,4,1
8,camera,True,3,1
9,ram,4,1,2


In [12]:
problem_constraint_list = []
unique_constraint_no = experiment_constraint_data['constraint_no'].unique()
for no in unique_constraint_no:
    constraint_list = []
    for index, item in experiment_constraint_data[experiment_constraint_data['constraint_no'] == no].iterrows():
        constraint:Constraint = Constraint(item.get('name'), item.get('value'), item.get('priority'), attribute_nature.get(item.get('name')))
        constraint_list.append(constraint)
    problem_constraint_list.append(constraint_list)

In [15]:
for constraint_list in problem_constraint_list:
    problem:Problem = Problem()
    print(problem.solve())

         brand                               model                      cpu  \
0    Microsoft   Surface Laptop Studio (ABY-00017)           Core i7-11370H   
99        Asus      Laptop 14 A416 (A416M-ABV550W)            Celeron N4020   
98         MSI                 Modern 15 (A5M-094)            Ryzen 7 5700U   
97      Lenovo              IdeaPad 3 (82H80256MJ)           Core i5-1135G7   
96         NaN                 Laptop 15S-FQ2511TU           Core i5-1135G7   
..         ...                                 ...                      ...   
46        Asus  ExpertBook B9450 (B9450F-ABM0285R)  Core i5-10210 processor   
47   Microsoft        Surface Laptop 4 (5EB-00018)           Core i7-1185G7   
48          HP    Pavilion Aero Laptop 13-be0045AU            Ryzen 5 5600U   
42      Lenovo            ThinkPad E14 Gen 2 (AMD)        Ryzen 5 PRO 4650U   
136        NaN   Surface Laptop Studio (A1Y-00017)           Core i7-11370H   

     ram  storage                                  